In [1]:
import numpy as np
import sys
from casadi import *

# Add do_mpc to path. This is not necessary if it was installed via pip
sys.path.append('../../../')

# Import do_mpc package:
import do_mpc

In [2]:
model_type = 'continuous' # either 'discrete' or 'continuous'
model = do_mpc.model.Model('continuous', 'MX')
# model = do_mpc.model.Model(model_type)

In [3]:
g = 9.80665 # m/s^2, Gravity

In [4]:
pos = model.set_variable('_x',  'pos',2)
dpos = model.set_variable('_x',  'dpos',2)


u = model.set_variable('_u',  'force',2)

ddpos = model.set_variable('_z', 'ddpos',2)
# prin

In [5]:
model.set_rhs('pos', dpos)
model.set_rhs('dpos', ddpos)
# model.set_rhs('time', 1)

In [6]:
formulas = vertcat(
        # 1
        u*0.001 + [0,-g] + ([50,50]-pos)
        - (ddpos),
    )

model.set_alg('formulas', formulas)

In [7]:

distance = (100-pos[1])**2 + (100 - pos[0])**2


model.set_expression('distance', distance)

MX((sq((100-pos[1]))+sq((100-pos[0]))))

In [8]:
# Build the model
model.setup()

In [9]:
mpc = do_mpc.controller.MPC(model)

In [10]:
setup_mpc = {
    'n_horizon': 100,
    'n_robust': 0,
    'open_loop': 0,
    't_step': 0.1,
    'state_discretization': 'collocation',
    'collocation_type': 'radau',
    'collocation_deg': 3,
    'collocation_ni': 1,
    'store_full_solution': True,
    # Use MA27 linear solver in ipopt for faster calculations:
    'nlpsol_opts': {'ipopt.linear_solver': 'mumps'}
}
mpc.set_param(**setup_mpc)

In [11]:
mterm = model.aux['distance'] # terminal cost
lterm = model.aux['distance'] # stage cost

mpc.set_objective(mterm=mterm, lterm=lterm)
# Input force is implicitly restricted through the objective.
mpc.set_rterm(force=0.1)

In [12]:
mpc.bounds['lower','_u','force'] = [-20,20]
mpc.bounds['upper','_u','force'] = [20,20]

In [13]:
mpc.setup()

In [14]:
estimator = do_mpc.estimator.StateFeedback(model)

In [15]:
simulator = do_mpc.simulator.Simulator(model)

In [16]:
params_simulator = {
    # Note: cvode doesn't support DAE systems.
    'integration_tool': 'idas',
    'abstol': 1e-10,
    'reltol': 1e-10,
    't_step': 0.04
}

simulator.set_param(**params_simulator)

In [17]:
simulator.setup()

In [18]:
simulator.x0['pos'] = [20,10]



x0 = simulator.x0.cat.full()

mpc.x0 = x0
estimator.x0 = x0

mpc.set_initial_guess()


In [19]:
import matplotlib.pyplot as plt
plt.ion()
from matplotlib import rcParams
rcParams['text.usetex'] = False
rcParams['axes.grid'] = True
rcParams['lines.linewidth'] = 2.0
rcParams['axes.labelsize'] = 'xx-large'
rcParams['xtick.labelsize'] = 'xx-large'
rcParams['ytick.labelsize'] = 'xx-large'

In [20]:
mpc_graphics = do_mpc.graphics.Graphics(simulator.data)
# print(pos)

In [21]:
%%capture

fig = plt.figure(figsize=(16,9))
fig.patch.set_facecolor('white')
plt.title("space ship")

ax1 = plt.subplot2grid((4, 2), (0, 0), rowspan=4)
ax2 = plt.subplot2grid((4, 2), (0, 1))
# ax3 = plt.subplot2grid((4, 2), (1, 1))
ax4 = plt.subplot2grid((4, 2), (2, 1))
# ax5 = plt.subplot2grid((4, 2), (3, 1))

ax2.set_ylabel('Distance to goal')
# ax3.set_ylabel('Posx ')
ax4.set_ylabel('Position x,y')
# ax5.set_ylabel('Input accel x,y')

# Axis on the right.
for ax in [ax2, ax4]:
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    if ax != ax5:
        ax.xaxis.set_ticklabels([])

ax5.set_xlabel('time [s]')

# ax1.axhline(0,color='black')
ax1.set_xlim(0,120)
ax1.set_ylim(0,120)

bar1 = ax1.plot([],[], '-o', linewidth=5, markersize=10)
bar2 = ax1.plot([],[], '-o', linewidth=5, markersize=10)

# mpc_graphics.add_line(var_type='_aux', var_name='d', axis=ax1)
mpc_graphics.add_line(var_type='_aux', var_name='distance', axis=ax2)
# mpc_graphics.add_line(var_type='_x', var_name='pos', axis=ax3)
mpc_graphics.add_line(var_type='_x', var_name='pos', axis=ax4)
mpc_graphics.add_line(var_type='_u', var_name='force', axis=ax5)


fig.align_ylabels()
fig.tight_layout()

NameError: name 'ax5' is not defined

In [22]:
# u0 = mpc.make_step(x0)

In [23]:


# mpc_graphics.plot_predictions()
# mpc_graphics.reset_axes()

# fig

In [24]:
# %%capture
# Quickly reset the history of the MPC data object.
mpc.reset_history()

n_steps = 100
u0 = mpc.make_step(x0)
print("~~~~~")
print(u0)
print(type(u0))
print("~~~~~")
for k in range(n_steps):
#     u0 = mpc.make_step(x0)
#     print("~~~~~")
#     print(u0)
#     print("~~~~~")
    y_next = simulator.make_step(np.array([[0],[0]]))
    x0 = estimator.make_step(y_next)
    


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     9604
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      401

Total number of variables............................:     2920
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      100
                     variables with only upper bounds:        0
Total number of equa

In [25]:
from matplotlib.animation import FuncAnimation, FFMpegWriter, ImageMagickWriter

# The function describing the gif:
# x = mpc.data['_x']
x = simulator.data['_x']
print(x)
# print(list(map(lambda n: [n[0]/100,n[1]/100], x)))
def update(t_ind):

    bar1[0].set_data(x[t_ind][0],x[t_ind][1])
    bar2[0].set_data(100,100)
    mpc_graphics.plot_results(t_ind)
    mpc_graphics.plot_predictions(t_ind)
    mpc_graphics.reset_axes()


anim = FuncAnimation(fig, update, frames=n_steps, repeat=False)
gif_writer = ImageMagickWriter(fps=20)
anim.save('anim.gif', writer=gif_writer)

MovieWriter stderr:
magick.exe: unexpected end-of-file '-': No such file or directory @ error/rgb.c/ReadRGBImage/249.



[[ 20.          10.           0.           0.        ]
 [ 20.0239968   10.02415146   1.19968003   1.20741196]
 [ 20.09594881  10.0965672    2.39744082   2.41289232]
 [ 20.21574092  10.21713137   3.59136622   3.61451257]
 [ 20.38318149  10.3856511    4.7795462    4.81035037]
 [ 20.59800266  10.60185679   5.96007992   5.99849264]
 [ 20.85986075  10.86540256   7.13107879   7.17703859]
 [ 21.16833685  11.17586678   8.29066945   8.34410281]
 [ 21.52293746  11.53275279   9.43699681   9.49781825]
 [ 21.92309529  11.93548964  10.56822698  10.6363392 ]
 [ 22.36817018  12.38343304  11.68255025  11.75784429]
 [ 22.85745011  12.87586637  12.77818393  12.86053933]
 [ 23.39015232  13.41200185  13.85337525  13.94266025]
 [ 23.96542461  13.99098177  14.90640411  15.00247589]
 [ 24.58234667  14.61187989  15.93558592  16.03829077]
 [ 25.23993155  15.27370291  16.93927418  17.0484478 ]
 [ 25.93712725  15.97539204  17.91586322  18.03133096]
 [ 26.67281843  16.71582474  18.86379069  18.98536782]
 [ 27.4458

CalledProcessError: Command '['J:\\devtools\\ImageMagick-7.1.0-Q16-HDRI\\magick.exe', '-size', '1152x648', '-depth', '8', '-delay', '5.0', '-loop', '0', 'rgba:-', 'anim.gif']' returned non-zero exit status 1.

In [ ]:
from IPython.display import HTML, Image
display(Image(data=open("anim.gif",'rb').read(), format='png'))

In [ ]:
# %%capture
# # Quickly reset the history of the MPC data object.
# mpc.reset_history()

# n_steps = 100
# for k in range(n_steps):
#     u0 = mpc.make_step(x0)
#     y_next = simulator.make_step(u0)
#     x0 = estimator.make_step(y_next)